# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../output_data/cities.csv')
cities.drop(['Unnamed: 0'], axis=1,inplace=True)
cities

,City,Temperature_max,Wind,Humidity,Latitude,Longitude,Cloudiness,Country
0,bambous virieux,25.00,3.60,94,-20.3428,57.7575,75,MU
1,butaritari,28.16,4.78,76,3.0707,172.7902,82,KI
2,general pico,12.65,3.48,77,-35.6566,-63.7568,0,AR
3,champerico,28.00,3.09,78,14.3000,-91.9167,75,GT
4,komarichi,-1.40,3.23,82,52.4151,34.7905,94,RU
...,...,...,...,...,...,...,...,...
584,kamina,18.70,1.25,92,-8.7386,24.9906,100,CD
585,macas,19.64,0.97,98,-2.3167,-78.1167,100,EC
586,kokkola,0.56,1.03,93,63.8385,23.1307,0,FI
587,karaman,7.16,4.77,78,37.0833,33.2500,47,TR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
tempf = []
for temp in cities['Temperature_max']:
    tempf.append(temp * 1.8000+ 32.00)
tempf
cities['Temperature_max_F'] = tempf
cities.head()


,City,Temperature_max,Wind,Humidity,Latitude,Longitude,Cloudiness,Country,Temperature_max_F
0,bambous virieux,25.00,3.60,94,-20.3428,57.7575,75,MU,77.000
1,butaritari,28.16,4.78,76,3.0707,172.7902,82,KI,82.688
2,general pico,12.65,3.48,77,-35.6566,-63.7568,0,AR,54.770
3,champerico,28.00,3.09,78,14.3000,-91.9167,75,GT,82.400
4,komarichi,-1.40,3.23,82,52.4151,34.7905,94,RU,29.480


In [4]:
gmaps.configure(api_key=g_key)

In [5]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [6]:

locations = cities[["Latitude", "Longitude"]].astype(float)
humid = cities['Humidity'].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
cities_ideal = cities.loc[(cities['Temperature_max_F'] < 80) & (cities['Temperature_max_F'] > 70) & (cities['Wind'] < 10) & (cities['Cloudiness'] == 0)]
cities_ideal.dropna(inplace=True)
cities_ideal.Humidity.describe()


<ipython-input-13-af5b2b2a4633>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_ideal.dropna(inplace=True)


count     14.000000
mean      65.642857
std       21.062570
min       26.000000
25%       58.750000
50%       72.500000
75%       74.000000
max      100.000000
Name: Humidity, dtype: float64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
target_coordinates = '-23.7000, 133.8833'
target_radius = 5000
target_search = "hotel"
params = {
"location": target_coordinates,
"radius": target_radius,
"keyword": target_search,
"key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params)
hotel_json = response.json()
hotel_json["results"][0]["plus_code"]["compound_code"]

'7VMH+XM Alice Springs, Northern Territory, Australia'

In [15]:
    lats.append(hotel_json['results'][0]['geometry']['location']['lat'])
    lngs.append(hotel_json['results'][0]['geometry']['location']['lng'])
    hotels.append(hotel_json["results"][0]["name"])

NameError: name 'lats' is not defined

In [16]:
hotel_df = pd.DataFrame()
hotel_df[["Lat", "Lng", 'Hotel Name']] = ''
hotels = []
lats = []
lngs = []
cities23 = []
country23 = []
for index, row in cities_ideal.iterrows():
    try:
        target_coordinates = f'{row[4]}, {row[5]}'
        target_radius = 5000
        target_search = "hotel"
        params = {
        "location": target_coordinates,
        "radius": target_radius,
        "keyword": target_search,
        "key": g_key
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        hotel_json = response.json()
        lats.append(hotel_json["results"][0]["geometry"]["location"]["lat"])
        lngs.append(hotel_json["results"][0]["geometry"]["location"]["lng"])
        hotels.append(hotel_json["results"][0]["name"]) 
        cities23.append(row[0])
        country23.append(row[7])
    except:
        print(f'No hotel found')

No hotel found
No hotel found


In [17]:
hotel_df["Hotel Name"] = hotels    
hotel_df["Lat"] = lats  
hotel_df["Lng"] = lngs
hotel_df['City'] = cities23
hotel_df['Country'] = country23
hotel_df

,Lat,Lng,Hotel Name,City,Country
0,29.945274,40.181156,Raoum Inn Hotel,sakakah,SA
1,26.545393,-77.049019,Abaco Beach Resort,marsh harbour,BS
2,24.082330,32.887831,Sofitel Legend Old Cataract Aswan,aswan,EG
3,-16.373373,-49.491872,Hotel Tarumã,inhumas,BR
4,14.844358,74.098633,Cintacor Island Resort,karwar,IN
5,-31.429814,152.916753,ibis Styles Port Macquarie,port macquarie,AU
6,-10.709754,-48.407689,Vila Real Hotel,porto nacional,BR
7,-15.863311,-38.875494,Pousada do Diogo,belmonte,BR
8,26.558308,101.727573,Jinhai New Century Grand Hotel,panzhihua,CN
9,-34.562229,150.830536,Lakeview Hotel Motel,flinders,AU


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))